In [1]:
import warnings
warnings.filterwarnings(action="ignore")
import datetime
import time
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

In [2]:
RANDOM_SEED = 99

In [3]:
train_df = pd.read_parquet('./data/train_v2_3_nn.parquet')
train_null_df = pd.read_parquet('./data/train_v2_3_n.parquet').drop(columns=["BN"])
test_df = pd.read_parquet('./data/test_v2_3.parquet')

In [4]:
train_x = train_df.drop(columns=["BN"])
train_y = train_df["BN"]

In [5]:
def create_model(param):
    return xgb.XGBRegressor(
        random_state=RANDOM_SEED,
        tree_method="gpu_hist",
        eval_metric="rmse",
        early_stopping_rounds=100,
        **param
    )

In [6]:
def train_model(param, X, y, X_test, trial=None, test=False):
  skf = KFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)

  val_scores = []
  y_tests = []
  models = []

  for idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):

    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

    model = create_model(param)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=1000)

    y_hat_val = model.predict(X_val)
    score = mean_squared_error(y_val.values, y_hat_val, squared=False)
    val_scores.append(score)
    print(f'Fold: {idx+1}/3 rmse score = {score:.5f}')

    if test:
      y_tests.append(model.predict(X_test))
      models.append(model)

    if trial:
      trial.report(score, idx)

      if trial.should_prune():
        raise optuna.TrialPruned()

  return val_scores, y_tests, models

In [7]:
def objective_xgb(trial):
  param = {
    "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
    "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
    "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    "subsample": trial.suggest_float("subsample", 0.1, 1.0),
    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
    "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
    "max_depth": trial.suggest_int("max_depth", 3, 9),
    "min_child_weight": trial.suggest_int("min_child_weight", 2, 10),
    "eta": trial.suggest_float("eta", 1e-8, 1.0, log=True),
    "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
    "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
  }

  if param["booster"] == "dart":
    param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
    param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
    param['rate_drop'] = trial.suggest_float('rate_drop', 1e-8, 1.0, log=True)
    param['skip_drop'] = trial.suggest_float('skip_drop', 1e-8, 1.0, log=True)

  val_scores, y_tests, models = train_model(param, train_x, train_y, test_df, trial)

  return sum(val_scores) / len(val_scores)

In [8]:
study = optuna.create_study(
    sampler=TPESampler(seed=RANDOM_SEED),
    direction='minimize',
    study_name='xgb_tuning',
    pruner=HyperbandPruner(
      min_resource=1, max_resource=8, reduction_factor=3
    ),
)

study.optimize(objective_xgb, n_trials=50)

[I 2023-10-10 15:08:45,536] A new study created in memory with name: xgb_tuning


[0]	validation_0-rmse:2.60828	validation_1-rmse:2.60307
[299]	validation_0-rmse:2.60827	validation_1-rmse:2.60307
Fold: 1/3 rmse score = 2.60307
[0]	validation_0-rmse:2.60492	validation_1-rmse:2.60978
[299]	validation_0-rmse:2.60492	validation_1-rmse:2.60978
Fold: 2/3 rmse score = 2.60978
[0]	validation_0-rmse:2.60643	validation_1-rmse:2.60677
[299]	validation_0-rmse:2.60643	validation_1-rmse:2.60677


[I 2023-10-10 15:08:58,338] Trial 0 finished with value: 2.606540033674593 and parameters: {'booster': 'gbtree', 'lambda': 0.04017550942202233, 'alpha': 1.7847241527490656e-08, 'subsample': 0.8272449670283629, 'colsample_bytree': 0.6090556776494775, 'n_estimators': 300, 'max_depth': 3, 'min_child_weight': 10, 'eta': 1.1339812255777714e-08, 'gamma': 0.014399394343769443, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 2.606540033674593.


Fold: 3/3 rmse score = 2.60677
[0]	validation_0-rmse:2.60827	validation_1-rmse:2.60307
[899]	validation_0-rmse:2.60448	validation_1-rmse:2.59929
Fold: 1/3 rmse score = 2.59929
[0]	validation_0-rmse:2.60492	validation_1-rmse:2.60978
[899]	validation_0-rmse:2.60114	validation_1-rmse:2.60599
Fold: 2/3 rmse score = 2.60599
[0]	validation_0-rmse:2.60643	validation_1-rmse:2.60677
[899]	validation_0-rmse:2.60264	validation_1-rmse:2.60298


[I 2023-10-10 15:41:45,624] Trial 1 finished with value: 2.602753930514485 and parameters: {'booster': 'dart', 'lambda': 1.4575798463318516e-05, 'alpha': 0.6189425995687715, 'subsample': 0.5719732438377161, 'colsample_bytree': 0.1842517839679773, 'n_estimators': 900, 'max_depth': 4, 'min_child_weight': 6, 'eta': 2.178476742104051e-06, 'gamma': 0.033817296080697445, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 1.4193450623251104e-07, 'skip_drop': 4.983900949448835e-07}. Best is trial 1 with value: 2.602753930514485.


...

Fold: 3/3 rmse score = 0.90972
[0]	validation_0-rmse:1.17469	validation_1-rmse:1.18732
[120]	validation_0-rmse:0.64092	validation_1-rmse:1.07874
Fold: 1/3 rmse score = 1.00295
[0]	validation_0-rmse:1.17861	validation_1-rmse:1.18211
[130]	validation_0-rmse:0.63398	validation_1-rmse:1.06217
Fold: 2/3 rmse score = 0.99671
[0]	validation_0-rmse:1.17877	validation_1-rmse:1.17827
[130]	validation_0-rmse:0.62642	validation_1-rmse:1.08654


[I 2023-10-10 18:20:36,097] Trial 48 finished with value: 1.00173026520643 and parameters: {'booster': 'gbtree', 'lambda': 0.000709666261209232, 'alpha': 2.6975470721312634e-08, 'subsample': 0.9409657636496274, 'colsample_bytree': 0.7196911784368303, 'n_estimators': 700, 'max_depth': 8, 'min_child_weight': 3, 'eta': 0.9129775065590999, 'gamma': 0.0002837258089877021, 'grow_policy': 'depthwise'}. Best is trial 43 with value: 0.8957313382356813.


Fold: 3/3 rmse score = 1.00553
[0]	validation_0-rmse:2.60359	validation_1-rmse:2.59841
[799]	validation_0-rmse:1.08404	validation_1-rmse:1.10512
Fold: 1/3 rmse score = 1.10512
[0]	validation_0-rmse:2.60025	validation_1-rmse:2.60511
[799]	validation_0-rmse:1.08901	validation_1-rmse:1.10255


[I 2023-10-10 18:21:16,943] Trial 49 pruned. 


Fold: 2/3 rmse score = 1.10255


In [9]:
trial = study.best_trial

In [10]:
print(trial.params)

{'booster': 'gbtree', 'lambda': 0.0031212004487157427, 'alpha': 1.3125027149309643e-08, 'subsample': 0.9226240655739183, 'colsample_bytree': 0.5563709697372726, 'n_estimators': 800, 'max_depth': 8, 'min_child_weight': 6, 'eta': 0.10070284604196998, 'gamma': 0.0024013475278437646, 'grow_policy': 'depthwise'}


In [11]:
best_param = {
  'booster': 'gbtree',
  'lambda': 0.0031212004487157427,
  'alpha': 1.3125027149309643e-08,
  'subsample': 0.9226240655739183,
  'colsample_bytree': 0.5563709697372726,
  'n_estimators': 800,
  'max_depth': 8,
  'min_child_weight': 6,
  'eta': 0.10070284604196998,
  'gamma': 0.0024013475278437646,
  'grow_policy': 'depthwise'
}

val_scores, y_tests, models = train_model(best_param, train_x, train_y, train_null_df, test=True)

[0]	validation_0-rmse:2.40770	validation_1-rmse:2.40370
[799]	validation_0-rmse:0.64490	validation_1-rmse:0.89985
Fold: 1/3 rmse score = 0.89983
[0]	validation_0-rmse:2.40510	validation_1-rmse:2.40954
[799]	validation_0-rmse:0.64698	validation_1-rmse:0.88969
Fold: 2/3 rmse score = 0.88967
[0]	validation_0-rmse:2.40622	validation_1-rmse:2.40644
[799]	validation_0-rmse:0.64386	validation_1-rmse:0.89772
Fold: 3/3 rmse score = 0.89770


In [12]:
for idx, model in enumerate(models):
  file_name = f"./models/BN_model_{idx}.model"
  model.save_model(file_name)

In [13]:
train_null_df["BN"] = np.mean(y_tests, axis=0)

In [14]:
result = pd.concat([train_df, train_null_df], axis=0).sort_index()

In [15]:
result.head(10)

,ARI_CO_BR,ARI_CO_CA,ARI_CO_CL,ARI_CO_CN,ARI_CO_FI,ARI_CO_ID,ARI_CO_IN,ARI_CO_JP,ARI_CO_KR,ARI_CO_LV,...,SHIP_TYPE_CATEGORY_Container,SHIP_TYPE_CATEGORY_Tanker,WITlt0,WTI,day,hour,minute,month,weekday,year
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,-1.081668,15,4,3,10,3,2020
1,0,0,0,1,0,0,0,0,0,0,...,1,0,0,-0.188614,17,2,55,9,1,2019
2,0,0,0,1,0,0,0,0,0,0,...,1,0,0,-0.305226,23,6,43,2,5,2019
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,-1.074380,18,22,6,9,4,2020
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1.209272,13,12,57,8,5,2022
5,0,0,0,1,0,0,0,0,0,0,...,1,0,0,-0.840670,8,14,24,9,1,2015
6,0,0,0,0,0,0,0,1,0,0,...,1,0,0,-0.507840,25,21,34,1,0,2021
7,0,0,0,0,0,0,0,1,0,0,...,0,0,0,-0.459737,18,3,48,6,1,2019
8,0,0,0,0,0,0,0,1,0,0,...,1,0,0,-0.488404,27,6,23,1,4,2017
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.008168,3,7,8,2,5,2018


In [16]:
result.to_parquet("./data/train_v2_2bn.parquet")

In [20]:
test_df = pd.read_parquet('./data/test_v2_3.parquet')
bn_null_idx = test_df[test_df["BN"].isna()].index
test_null_df = test_df.loc[bn_null_idx]
bn_idx = test_df[test_df["BN"].notna()].index
test_df = test_df.loc[bn_idx]

In [22]:
print(len(test_df))
print(len(test_null_df))

143062
101927


In [23]:
test_null_df = test_null_df.drop(columns=["BN"])

In [24]:
test_winds = []
for model in models:
  test_winds.append(model.predict(test_null_df))

In [25]:
test_null_df["BN"] = np.mean(test_winds, axis=0)

In [26]:
assert sorted(train_df.columns) == sorted(test_df.columns)

In [27]:
test_null_df.head()

,ARI_CO_BR,ARI_CO_CA,ARI_CO_CL,ARI_CO_CN,ARI_CO_FI,ARI_CO_ID,ARI_CO_IN,ARI_CO_JP,ARI_CO_KR,ARI_CO_LV,...,SHIP_TYPE_CATEGORY_Tanker,WITlt0,WTI,day,hour,minute,month,weekday,year,BN
3,0,0,0,0,0,0,0,1,0,0,...,0,0,-1.152121,2,0,59,8,1,2016,0.606377
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0.269088,14,10,6,7,5,2018,2.777071
6,0,0,0,0,0,0,0,0,0,0,...,0,0,-0.524846,18,14,11,1,2,2017,0.690901
7,0,0,0,0,0,0,0,0,0,0,...,0,0,-0.613276,12,0,57,10,3,2017,1.149996
11,0,0,0,0,0,0,0,0,0,0,...,0,0,-1.337242,2,6,6,4,5,2016,2.351849


In [28]:
test_result = pd.concat([test_df, test_null_df], axis=0).sort_index()

In [30]:
test_result.head(10)

,ARI_CO_BR,ARI_CO_CA,ARI_CO_CL,ARI_CO_CN,ARI_CO_FI,ARI_CO_ID,ARI_CO_IN,ARI_CO_JP,ARI_CO_KR,ARI_CO_LV,...,SHIP_TYPE_CATEGORY_Container,SHIP_TYPE_CATEGORY_Tanker,WITlt0,WTI,day,hour,minute,month,weekday,year
0,0,0,0,0,0,0,0,1,0,0,...,0,1,0,1.279239,27,8,29,8,5,2022
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1.816140,27,20,57,3,6,2022
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.789955,18,2,6,1,2,2023
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,-1.152121,2,0,59,8,1,2016
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0.821537,24,0,27,1,1,2023
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.269088,14,10,6,7,5,2018
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-0.524846,18,14,11,1,2,2017
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-0.613276,12,0,57,10,3,2017
8,0,0,0,0,0,0,0,1,0,0,...,0,0,0,-0.087551,13,0,35,3,5,2021
9,0,0,0,0,0,0,0,1,0,0,...,1,0,0,-0.985949,29,5,46,7,2,2020


In [31]:
test_df.to_parquet("./data/test_v2_3.parquet")